In [1]:
import os

import matplotlib
import numpy as np
import librosa.display
import IPython.display as ipd
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras import models
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D,Conv1D, Flatten,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import seaborn as sn
import random

prog = []
nonprog = []
total_files = 0
total_prog = 0
total_non_prog = 0

# find total files
for root, dirs, files in os.walk("CAP6610SP24_training_set/Progressive_Rock_Songs", topdown=False):
   for name in files:
      filename = os.path.join(root, name)
      if filename.find(".mp3") != -1 or filename.find(".wav") != -1 or filename.find(".flac") != -1 or filename.find(".avi") != -1 or filename.find(".m4a") != -1 or filename.find(".ogg") != -1 :
          total_files = total_files + 1
          total_prog += 1 

for root, dirs, files in os.walk("CAP6610SP24_training_set/Not_Progressive_Rock", topdown=False):
   for name in files:
      filename = os.path.join(root, name)
      if filename.find(".mp3") != -1 or filename.find(".wav") != -1 or filename.find(".flac") != -1 or filename.find(".avi") != -1 or filename.find(".m4a") != -1 or filename.find(".ogg") != -1 :
          total_files = total_files + 1
          total_non_prog += 1



# --------Extract data from csv file--------------------------------------

data = pd.read_csv('training_features_10.csv')
data.head()            
new_files = data['filename']
new_files = list(new_files)
for i in range(len(new_files)) :
    new_files[i] = (new_files[i].split("chunk") )[0]


data = data.drop(['filename'],axis=1)
data = data.drop(['mfcc14'],axis=1)
data = data.drop(['mfcc19'],axis=1)
data = data.drop(['mfcc9'],axis=1)
data = data.drop(['mfcc12'],axis=1)
data = data.drop(['mfcc20'],axis=1)
data = data.drop(['chroma_stft'],axis=1)

data = np.asarray(data)

# X = data[:,6:]
X = data[:,1:]
y = data[:,0]
y[y=="prog"] = 0
y[y=="non_prog"] = 1
y[y=="djent"] = 0

X = np.reshape(X, (X.shape[0],1, X.shape[1]))

print("X_train shape ",X.shape)
y_test = y
y_test = to_categorical(y_test)
 

model = load_model('Model_best.h5')
# model = load_model('Model.h5')

print("\nTesting ...")

# Predict using saved model

X= X.astype('float32')

predictions = model.predict(X)
print(predictions)
row,col = predictions.shape
count = 0
for i in range(row) :
    true_val = np.argmax(y_test[i])
    predicted_val = np.argmax(predictions[i]) 
    # print("true ",true_val,"    got ",predicted_val)
    if true_val == predicted_val :
       count += 1
print("Accuracy on 10s snippets",count," / ",row,'-------' ,(count/row) *100," %")
print("Total files ",total_files)


new_true_ind = []
new_predicted_ind = []
i = 0

ind = []     
start = 0
for i in range(len(new_files)) :
    if i == 0 :
        start = 0
    elif i == len(new_files) - 1 :
        if new_files[i] == new_files[i-1] :
            ind.append([new_files[i],start,len(new_files)-1])  
        else :
            ind.append([new_files[i],len(new_files)-1,len(new_files)-1])  

    else :
         if new_files[i] != new_files[i-1] :
             end = i-1
             ind.append([new_files[i-1],start,end])  
             start = i
    
print("indices calculated") 

for i in range(row) :
    new_predicted_ind.append(np.argmax(predictions[i]))
    new_true_ind.append(np.argmax(y_test[i]))

# Find majority result for song chunks

def calc_majority(indices,predicted,true) :
    name = indices[0]
    start = indices[1]
    end = indices[2]
    new_predicted = predicted[start:end+1]
    new_true = true[start:end+1]
    maj_pred = 0
    maj_true = 0
    for i in range(len(new_predicted)) :
        if new_predicted[i] == 0 :
            maj_pred += 1
    if maj_pred <= 2*len(new_predicted)/3  :       
       ret_pred_val = 1
    else:
        ret_pred_val=0

    if name.find("nonprog") != -1 :
        ret_true_val = 1
    else :
        ret_true_val = 0        
    return [ret_pred_val,ret_true_val, name]



majority_pred = []
majority_true = []
majority_true_name = []

print("len ind",len(ind))

# Find accuracy for all chunks by checking majority output
for i in range(len(ind)) :
    ret_pred_val,ret_true_val, name = calc_majority(ind[i],new_predicted_ind,new_true_ind)
    majority_pred.append(ret_pred_val)
    majority_true.append(ret_true_val)
    majority_true_name.append(name)
count = 0
# print("len maj",len(majority_pred))
for i in range(len(majority_pred)) :
    # print(majority_true_name[i], "------------ true ",majority_true[i],"     predicted",majority_pred[i])
    if majority_pred[i] == majority_true[i] :
        count += 1

print("Accuracy on songs",count," / ",total_files, (count/total_files) *100," %" )

# create a new dataframe with majority results
df = pd.DataFrame(majority_true_name, columns = ['filename'])
df['true'] = majority_true
df['predicted'] = majority_pred
# shuffle entriess
df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv('final_results_train.csv',index=False)
print(df.head())
# Plot confusion matrix

confusion_matrix = confusion_matrix(majority_true, majority_pred ) 
df_cm = pd.DataFrame(confusion_matrix, index = [i for i in ["True prog songs","True nonprog songs"]],
                  columns = [i for i in ["Predicted prog songs","Predicted nonprog songs"]])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.title('Total prog = %i, Total nonprog = %i' %(total_prog,total_non_prog) )
plt.show()

X_train shape  (19605, 1, 20)

Testing ...
613/613 ━━━━━━━━━━━━━━━━━━━━ 1s 615us/step
[[0.69822717 0.25632372]
 [0.69122773 0.27483538]
 [0.6987361  0.2548814 ]
 ...
 [0.3570901  0.6496138 ]
 [0.3570907  0.64961296]
 [0.35617232 0.6342177 ]]
Accuracy on 10s snippets 15489  /  19605 ------- 79.0053557765876  %
Total files  501
indices calculated
len ind 501
Accuracy on songs 405  /  501 80.83832335329342  %
                                            filename  true  predicted
0                        nonprog04_-_Condor_Pasa.mp3     1          1
1        nonprog08_I_Could_Have_Danced_All_Night.mp3     1          1
2            nonprog1993_Snap_Rhythm_Is_A_Dancer.mp3     1          0
3            nonprog2015_Rihanna_FourFiveSeconds.mp3     1          1
4  prog01_2112_I_Overture_II_Temples_of_Syrinx_II...     0          0


In [2]:
df.head(15)

,filename,true,predicted
0,nonprog04_-_Condor_Pasa.mp3,1,1
1,nonprog08_I_Could_Have_Danced_All_Night.mp3,1,1
2,nonprog1993_Snap_Rhythm_Is_A_Dancer.mp3,1,0
3,nonprog2015_Rihanna_FourFiveSeconds.mp3,1,1
4,prog01_2112_I_Overture_II_Temples_of_Syrinx_II...,0,0
5,nonprog1991_Mariah_Carey_Someday.mp3,1,1
6,nonprog01_-_Under_the_Pressure.mp3,1,0
7,nonprog05-bangles-hazy_shade_of_winter.mp3,1,1
8,progThe_Flower_Kings_-_1997_Stardust_We_Are_CD...,0,0
9,prog04_Operation-_Mindcrime.mp3,0,1


In [3]:
# new dataframe where true ! = predicted
df_incorrect = df[df['true'] != df['predicted']]

df_incorrect.head(15)

,filename,true,predicted
2,nonprog1993_Snap_Rhythm_Is_A_Dancer.mp3,1,0
6,nonprog01_-_Under_the_Pressure.mp3,1,0
9,prog04_Operation-_Mindcrime.mp3,0,1
12,prog-04-_Knots.mp3,0,1
17,nonprogAli_Farka_Toure_with_Ry_Cooder_-_Talkin...,1,0
22,nonprog10-hozier-be-65228571.mp3,1,0
29,progCaravan_The_Love_In_Your_Eye.mp3,0,1
32,nonprog04-deacon_blues_320_lame_cbr.mp3,1,0
37,nonprog06_The_Temple_of_the_King.mp3,1,0
41,nonprog01_-_Mustt_Mustt_(Lost_in_His_Work).mp3,1,0
